# Aprendizado de conluio em licitações públicas
## Etapas do pré-processamento

## ver qual atributo selecionado na primeira execucao (ano ou mes)

In [1]:
#!pip install pyodbc
#!pip install mlxtend
#!pip install xlrd
#!pip install lime
#!pip install shap
#!pip install imblearn
#!pip install seaborn
#!pip install psycopg2
#!pip install category_encoders
#!pip install xgboost

In [2]:
import pandas as pd
import gc

## Carregar dados de licitações do governo Federal

#### Fonte: https://portaldatransparencia.gov.br/download-de-dados/licitacoes

### Carregar dados - Licitações

In [3]:
import os

In [4]:
def get_files_dir (pasta, ano):
    df = pd.DataFrame()
    for a in os.listdir(pasta):
        arquivo = pasta+'\\'+a
        if os.path.isfile(arquivo):     
            df_t = pd.read_csv(arquivo, sep=';', encoding='utf-8', low_memory=False)
            df = pd.concat([df, df_t], ignore_index=True)
        elif a == ano:
            df_t = get_files_dir (arquivo, ano)
            df = pd.concat([df, df_t], ignore_index=True)
    return df 

In [5]:
def get_files (dir, ano):   
    df = get_files_dir (dir, ano)
    df = df.drop_duplicates(ignore_index=True)
    #Só faz sentido avaliar quem tem concorrente
    df = df.loc[(df['Modalidade Compra'] != "Inexigibilidade de Licitação") & (df['Modalidade Compra'] != "Dispensa de Licitação")]
    return df

In [6]:
#Carregar a partir dos arquivos
def carregar_dados (anos):
    dir_licit = 'C:\\Users\\lucas\\Downloads\\Cefet-RJ\\Licitacao'
    dir_item = 'C:\\Users\\lucas\\Downloads\\Cefet-RJ\\ItemLicitacao'
    dir_partic = 'C:\\Users\\lucas\\Downloads\\Cefet-RJ\\ParticipantesLicitacao'
    df_u = pd.DataFrame()
    for a in anos:
        print ('Ano: '+a)
        df_item = get_files (dir_item, a)
        print ('Total Item: ', df_item.shape)
        df_partic = get_files (dir_partic, a)
        print ('Total Participantes: ',df_partic.shape)
        df_t = df_partic.merge(df_item, left_index=False, right_index=False, how="inner")
        df_licit = get_files (dir_licit, a)
        print ('Total Licitações: ',df_licit.shape)
        df_t = df_t.merge(df_licit, left_index=False, right_index=False, how="inner")
        print ('Total: ',df_t.shape)
        del(df_item, df_partic, df_licit)
        n = gc.collect()
        #Antecipa exclusão de colunas código (redundantes) para evitar problemas de memória
        cols = [x for x in df_t.columns if 'Código' in x]
        df_t.drop(columns=cols, inplace=True)
        df_u = pd.concat([df_u, df_t], ignore_index=True)
        del(df_t)
    print ('Total')
    print (df_u.shape)
    return df_u

In [7]:
anos = ['2013', '2014', '2015', '2016', '2017', '2018']

In [8]:
df_u = carregar_dados(anos)

Ano: 2013
Total Item:  (1202710, 14)
Total Participantes:  (8760019, 13)
Total Licitações:  (46616, 17)
Total:  (8606933, 27)
Ano: 2014
Total Item:  (1205338, 14)
Total Participantes:  (8293507, 13)
Total Licitações:  (46187, 17)
Total:  (8169073, 27)
Ano: 2015
Total Item:  (1022420, 14)
Total Participantes:  (7176613, 13)
Total Licitações:  (36997, 17)
Total:  (7076919, 27)
Ano: 2016
Total Item:  (1040179, 14)
Total Participantes:  (6873577, 13)
Total Licitações:  (37121, 17)
Total:  (6794974, 27)
Ano: 2017
Total Item:  (978603, 14)
Total Participantes:  (6846230, 13)
Total Licitações:  (34716, 17)
Total:  (6784984, 27)
Ano: 2018
Total Item:  (933067, 14)
Total Participantes:  (6416964, 13)
Total Licitações:  (33331, 17)
Total:  (6363128, 27)
Total
(43796011, 20)


In [9]:
df_u.head(50)

,Número Licitação,Nome UG,Modalidade Compra,Número Processo,Nome Órgão,Descrição Item Compra,Nome Participante,Flag Vencedor,Descrição,Quantidade Item,Valor Item,Nome Vencedor,Objeto,Situação Licitação,Nome Órgão Superior,UF,Município,Data Resultado Compra,Data Abertura,Valor Licitação
0,12010,26 GRUPO DE ARTILHARIA DE CAMPANHA,Tomada de Preços,001,Comando do Exército,CANTINA / BAR / LANCHONETE / REFEICAO RAPIDA /...,CONTRATADO : GILDA DE FATIMA DOS SANTOS - ME -,SIM,CANTINA / BAR / LANCHONETE / REFEICAO RAPIDA /...,1,"1158,3400",CONTRATADO : GILDA DE FATIMA DOS SANTOS - ME -,Objeto: Cessão de dependências interna para pr...,Evento de Resultado de Julgame,Ministério da Defesa,PR,GUARAPUAVA,24/01/2013,NaN,"1158,3400"
1,12012,UNIVERSIDADE FEDERAL RURAL DA AMAZONIA,Concorrência,23084.007315/2012,Universidade Federal Rural da Amazônia,OBRAS CIVIS PUBLICAS ( CONSTRUCAO ),R M H ENGENHARIA LTDA,SIM,OBRAS CIVIS PUBLICAS ( CONSTRUCAO ),1,"4278450,8000",R M H ENGENHARIA LTDA,Objeto: Contratação de empresa(s) especializad...,Evento de Alteração de Resulta,Ministério da Educação,PA,BELEM,15/01/2013,NaN,"6588848,3500"
2,12012,UNIVERSIDADE FEDERAL RURAL DA AMAZONIA,Concorrência,23084.007315/2012,Universidade Federal Rural da Amazônia,OBRAS CIVIS PUBLICAS ( CONSTRUCAO ),R M H ENGENHARIA LTDA,SIM,OBRAS CIVIS PUBLICAS ( CONSTRUCAO ),1,"2217180,7900",R M H ENGENHARIA LTDA,Objeto: Contratação de empresa(s) especializad...,Evento de Alteração de Resulta,Ministério da Educação,PA,BELEM,15/01/2013,NaN,"6588848,3500"
3,12012,UNIVERSIDADE FEDERAL RURAL DA AMAZONIA,Concorrência,23084.007315/2012,Universidade Federal Rural da Amazônia,OBRAS CIVIS PUBLICAS ( CONSTRUCAO ),VIVA SERVICOS DE MAO DE OBRA LTDA,SIM,OBRAS CIVIS PUBLICAS ( CONSTRUCAO ),1,"93216,7600",VIVA SERVICOS DE MAO DE OBRA LTDA,Objeto: Contratação de empresa(s) especializad...,Evento de Alteração de Resulta,Ministério da Educação,PA,BELEM,15/01/2013,NaN,"6588848,3500"
4,12012,IFSULDEMINAS - CAMPUS INCONFIDENTES,Convite,23344000613201201,Instituto Federal do Sul de Minas Gerais,OBRAS CIVIS DE PAVIMENTACAO DE CONCRETO,UIRAPURU INDUSTRIA E COMERCIO DE PRE-MOLDADOS ...,SIM,OBRAS CIVIS DE PAVIMENTACAO DE CONCRETO,1,"38824,7700",UIRAPURU INDUSTRIA E COMERCIO DE PRE-MOLDADOS ...,Objeto: Contratação de empresa para execução d...,Evento de Resultado de Julgame,Ministério da Educação,MG,INCONFIDENTES,22/01/2013,NaN,"38824,7700"
5,12012,BASE DE APOIO LOGISTICO DO EXERCITO,Pregão - Registro de Preço,65261.000512/2012,Comando do Exército,COLETA DE LIXO - RESIDENCIAL / COMERCIAL / IND...,SANETRAN - SANEAMENTO AMBIENTAL LTDA,NÃO,COLETA DE LIXO - RESIDENCIAL / COMERCIAL / IND...,36,"53640,0000",DEPOSITO DE PAPEL SANTA CECILIA LTDA,Objeto: Pregão Eletrônico - Contratação de se...,Publicado,Ministério da Defesa,RJ,RIO DE JANEIRO,08/01/2013,04/12/2012,"2880,0000"
6,12012,BASE DE APOIO LOGISTICO DO EXERCITO,Pregão - Registro de Preço,65261.000512/2012,Comando do Exército,COLETA DE LIXO - RESIDENCIAL / COMERCIAL / IND...,ABORGAMA DO BRASIL LTDA,NÃO,COLETA DE LIXO - RESIDENCIAL / COMERCIAL / IND...,36,"53640,0000",DEPOSITO DE PAPEL SANTA CECILIA LTDA,Objeto: Pregão Eletrônico - Contratação de se...,Publicado,Ministério da Defesa,RJ,RIO DE JANEIRO,08/01/2013,04/12/2012,"2880,0000"
7,12012,BASE DE APOIO LOGISTICO DO EXERCITO,Pregão - Registro de Preço,65261.000512/2012,Comando do Exército,COLETA DE LIXO - RESIDENCIAL / COMERCIAL / IND...,ATUAL 2005 - COLETA DE RESIDUOS LTDA,NÃO,COLETA DE LIXO - RESIDENCIAL / COMERCIAL / IND...,36,"53640,0000",DEPOSITO DE PAPEL SANTA CECILIA LTDA,Objeto: Pregão Eletrônico - Contratação de se...,Publicado,Ministério da Defesa,RJ,RIO DE JANEIRO,08/01/2013,04/12/2012,"2880,0000"
8,12012,BASE DE APOIO LOGISTICO DO EXERCITO,Pregão - Registro de Preço,65261.000512/2012,Comando do Exército,COLETA DE LIXO - RESIDENCIAL / COMERCIAL / IND...,GELMAR COMERCIO E SERVICOS LTDA,NÃO,COLETA DE LIXO - RESIDENCIAL / COMERCIAL / IND...,36,"53640,0000",DEPOSITO DE PAPEL SANTA CECILIA LTDA,Objeto: Preg

In [10]:
#Padroniza nomes colunas
df_u.columns = df_u.columns.str.upper().str.replace(' ', '_')
df_u.columns = df_u.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [11]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

Número de objetos coletados pela função GC: 28
Objetos restantes: []


## Análise exploratória dos dados e Pré-processamento

### Informações gerais dos atributos

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
def escalar_df(df_t):
    df = df_t.copy()
    objList = df.select_dtypes(include = ['object', 'category']).columns
    labelencoder = LabelEncoder()
    for feat in objList:
        df[feat] = labelencoder.fit_transform(df[feat].astype(str))
    scaler = MinMaxScaler()
    scaler.fit(df)
    scaled = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled, columns=df.columns)
    return scaled_df

In [ ]:
#Avalia atributos com muitos valores únicos
df = df_u.applymap(str)
df.describe(include=[object])
del df

In [ ]:
#Avalia atributos com muitos valores nulos
print(df_u.isnull().sum())

In [ ]:
#Avalia atributos com baixa variância (contribuem pouco para o aprendizado)
df_u_scaled = escalar_df(df_u)
print(df_u_scaled.var())

In [ ]:
#Avalia tipo dos atributos
print(df_u.dtypes)

### Seleção de atributos, transformações e limpezas

In [ ]:
#Descrição = Descrição Item
del df_u['DESCRICAO']
#Muitos valores vazios
del df_u['DATA_ABERTURA']
#Muitos valores únicos
del df_u['OBJETO']
del df_u['NUMERO_PROCESSO']
#Nome Órgão Superior tem baixa variabilidade, além disso, mais importante a identificação mais granular (Nome UG)
del df_u['NOME_ORGAO_SUPERIOR']
del df_u['NOME_ORGAO']
#Nome do vencedor já é um campo, pra que indicar se ele venceu?
del df_u['FLAG_VENCEDOR']
#Mais relevante será considerar o valor individual do item
del df_u['VALOR_LICITACAO']

In [ ]:
df_u['RESULTADO_COMPRA_MES'] = df_u['DATA_RESULTADO_COMPRA'].apply(lambda x : x[3:5])
df_u['RESULTADO_COMPRA_ANO'] = df_u['DATA_RESULTADO_COMPRA'].apply(lambda x : x[6:10])
del df_u['DATA_RESULTADO_COMPRA']

In [ ]:
df_u['MUNICIPIO'] = df_u['MUNICIPIO'] + "-" + df_u['UF']
del df_u['UF']

In [ ]:
df_u['VALOR_ITEM'] = df_u['VALOR_ITEM'].apply(lambda x: x.replace(',', '.')).astype(float)
#Se existir casos de quantidade de item = 0, remove
df_u = df_u.loc[df_u['QUANTIDADE_ITEM'] > 0]
df_u['VALOR_ITEM_UNIT'] = df_u['VALOR_ITEM']/df_u['QUANTIDADE_ITEM']
# Retira coluna quantidade e valor de item porque determinam valor licitação
del df_u['QUANTIDADE_ITEM']
del df_u['VALOR_ITEM']

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
df_u['DESCRICAO_ITEM_COMPRA'] = df_u[['DESCRICAO_ITEM_COMPRA']].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
gc.collect()

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
df_u['NOME_PARTICIPANTE'] = df_u[['NOME_PARTICIPANTE']].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
gc.collect()

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
df_u['NOME_VENCEDOR'] = df_u[['NOME_VENCEDOR']].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
gc.collect()

In [ ]:
df_u = df_u.drop_duplicates()
gc.collect()

In [ ]:
print (df_u.shape)
print (df_u.columns)

In [ ]:
#BACKUP
df_u.to_csv('C:\\Users\\lucas\\Downloads\\Cefet-RJ\\licitacoes_tratadas.csv', sep=';', index=False)

In [ ]:
#CARREGA BACKUP
#Carregar dados, quando os passos anteriores já tiverem sido executados
df_u = pd.read_csv('C:\\Users\\lucas\\Downloads\\Cefet-RJ\\licitacoes_tratadas.csv', sep=';', encoding='utf-8', low_memory=True)
df_u['RESULTADO_COMPRA_ANO'] = df_u['RESULTADO_COMPRA_ANO'].astype(str)

## Rotula licitações com conluio

### Carregar dados - Conluios

In [ ]:
df_cc = pd.read_excel('G:\\Meu Drive\\Cefet-rj\\Projeto de pesquisa\\conluios.xlsx', index_col=None, sheet_name='conluios') 

In [ ]:
#Padroniza nomes colunas
df_cc.columns = df_cc.columns.str.upper().str.replace(' ', '_')
df_cc.columns = df_cc.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
#Padroniza valores
df_cc = df_cc.apply(lambda x: x.astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [ ]:
df_cc['NOME_PARTICIPANTE'] = df_cc[['NOME_PARTICIPANTE']].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [ ]:
df_cc.head(100)

In [ ]:
print(df_cc['PROCESSO_ADMINISTRATIVO_(CADE)'].drop_duplicates())

### Rotula conluios

In [ ]:
#Cria atributo pra marcar linhas que algum participante do conluio venceu
df_u.loc[df_u['NOME_VENCEDOR'].isin(df_cc['NOME_PARTICIPANTE']), 'VENCEDOR_COLUIO'] = 'SIM'
print (df_u.loc[df_u['VENCEDOR_COLUIO']== 'SIM'].shape)

In [ ]:
lst = df_cc['PROCESSO_ADMINISTRATIVO_(CADE)'].drop_duplicates()
#Itera sobre cada processo administrativo
for proc in lst:
    print('Processo administrativo: ', proc)
    df_ct = df_cc.loc[df_cc['PROCESSO_ADMINISTRATIVO_(CADE)'] == proc]
    #Obtém os participantes de cada processo administrativo de conluio
    df_ut = df_u.loc[df_u['NOME_PARTICIPANTE'].isin(df_ct['NOME_PARTICIPANTE'])]
    #Itera sobre cada participante do processo administrativo avaliado
    for index, row in df_ct.iterrows():
        #Cria atributo para marcar linhas que o registro tenha um participante condenado por conluio no período determinado pelo CADE
        df_ut.loc[(df_ut['NOME_PARTICIPANTE'] == row['NOME_PARTICIPANTE']) & (df_ut['RESULTADO_COMPRA_ANO'].isin(str(row['PERIODO']).split(', '))), 'PARTICIPA_CONLUIO_PERIODO'] = 'SIM'
        print("Total participantes", df_ut.shape[0])
        #Seleciona apenas os registros que tenha participantes condenados por conluio no período
        df_count = df_ut.loc[df_ut['PARTICIPA_CONLUIO_PERIODO'] == 'SIM']
        print("Quantidade participa conluio", df_count.shape[0])
        df_count = df_count[['NUMERO_LICITACAO', 'NOME_PARTICIPANTE', 'RESULTADO_COMPRA_ANO', 'NOME_VENCEDOR', 'VENCEDOR_COLUIO']].drop_duplicates()
        df_count['QTD_PARTICIPANTES_CONLUIO'] = df_count.groupby(by=['NUMERO_LICITACAO'])['NOME_PARTICIPANTE'].transform('nunique')
        #Seleciona as linhas que o vencedor faça parte do conluio e que tenha mais de um participante como parte do conluio
        df_count = df_count.loc[(df_count['QTD_PARTICIPANTES_CONLUIO'] > 1) & (df_count['VENCEDOR_COLUIO'] == 'SIM')]
        #Para cada linha identificada como conluio, classifica a linha correspondente no conjunto de dados
        for index2, row2 in df_count.iterrows():
            df_u.loc[(df_u['NUMERO_LICITACAO'] == row2['NUMERO_LICITACAO']) & (df_u['NOME_VENCEDOR'] == row2['NOME_VENCEDOR']) & (df_u['RESULTADO_COMPRA_ANO'] == row2['RESULTADO_COMPRA_ANO']), 'LICITACAO_CONLUIO'] = 'SIM'
        #Imprime a quantidade de conluios existentes para o processo
        if (df_count.shape[0] > 0):
            print("Conluios: ", df_u.loc[df_u['LICITACAO_CONLUIO'] == 'SIM'].shape[0])
        del(df_count)
    del(df_ut)
    del(df_ct)

In [ ]:
df_u.loc[df_u['LICITACAO_CONLUIO'] == 'SIM'].head(50)

In [ ]:
df_u.loc[df_u['LICITACAO_CONLUIO'].isna(), 'LICITACAO_CONLUIO'] = 'NAO'
#Elimina atributos utilizados para calculo da classificao, para não enviesar a determinação
df_u = df_u.drop(columns=['VENCEDOR_COLUIO'], axis=1)

In [ ]:
print (df_u.shape)
print (df_u.columns)

In [ ]:
print (df_u.loc[df_u['LICITACAO_CONLUIO']== 'SIM'].shape)
print (df_u.loc[df_u['LICITACAO_CONLUIO']== 'NAO'].shape)

### Seleciona apenas itens com relação ao conluio

In [ ]:
desc = df_u.loc[(df_u['LICITACAO_CONLUIO'] == 'SIM'), 'DESCRICAO_ITEM_COMPRA'].drop_duplicates().to_list()
print(desc)

In [ ]:
df_u = df_u.loc[df_u['DESCRICAO_ITEM_COMPRA'].isin(desc)]

In [ ]:
print (df_u.loc[df_u['LICITACAO_CONLUIO']== 'SIM'].shape)
print (df_u.loc[df_u['LICITACAO_CONLUIO']== 'NAO'].shape)

## Enriquecimento do conjunto de dados

### Acrescenta variáveis estatística

In [ ]:
import scipy.stats as stats

In [ ]:
#Calcular estatisticas do valor unitario do item
df_u['COEFICIENTE_VARIACAO_ITEM'] = df_u.groupby(['DESCRICAO_ITEM_COMPRA','LICITACAO_CONLUIO'])['VALOR_ITEM_UNIT'].transform(lambda x: stats.variation(x) if len(x) > 1 else 0)
df_u['CURTOSE_ITEM'] = df_u.groupby(['DESCRICAO_ITEM_COMPRA','LICITACAO_CONLUIO'])['VALOR_ITEM_UNIT'].transform(lambda x: stats.kurtosis(x) if len(x) > 1 else 0)
df_u['ASSIMETRIA'] = df_u.groupby(['DESCRICAO_ITEM_COMPRA','LICITACAO_CONLUIO'])['VALOR_ITEM_UNIT'].transform(lambda x: stats.skew(x) if len(x) > 1 else 0)
df_u['3_QUARTIL_ITEM'] = df_u.groupby(['DESCRICAO_ITEM_COMPRA'])['VALOR_ITEM_UNIT'].transform(lambda x: x.quantile(0.75) if len(x) > 1 else 0)
df_u['OUTLIER'] = df_u.apply(lambda x: 'SIM' if x['VALOR_ITEM_UNIT'] > x['3_QUARTIL_ITEM'] else 'NAO', axis=1)
df_u['QTD_PARTICIPANTES_ITEM'] = df_u.groupby(['NUMERO_LICITACAO', 'NOME_UG', 'DESCRICAO_ITEM_COMPRA'])['NOME_PARTICIPANTE'].transform('nunique')

In [ ]:
df_u.head()

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
df_u = df_u.drop_duplicates()

In [ ]:
print (df_u.loc[df_u['LICITACAO_CONLUIO']== 'SIM'].shape)
print (df_u.loc[df_u['LICITACAO_CONLUIO']== 'NAO'].shape)

In [ ]:
#BACKUP
df_u.to_csv('C:\\Users\\lucas\\Downloads\\Cefet-RJ\\licitacoes_conluios_tratadas.csv', sep=';', index=False)

In [ ]:
#CARREGA BACKUP
#Carregar dados, quando os passos anteriores já tiverem sido executados
df_u = pd.read_csv('C:\\Users\\lucas\\Downloads\\Cefet-RJ\\licitacoes_conluios_tratadas.csv', sep=';', encoding='utf-8', low_memory=True)
df_u['RESULTADO_COMPRA_ANO'] = df_u['RESULTADO_COMPRA_ANO'].astype(str)

In [ ]:
# Baixa variabilidade. Exclui após utilização para sumarização.
df_u.drop(columns='NUMERO_LICITACAO', axis=1, inplace=True)
#Exclui atributos utilizados para calcular outros
df_u = df_u.drop(columns=['3_QUARTIL_ITEM', 'VALOR_ITEM_UNIT'], axis=1)

In [ ]:
df_u = df_u.reindex(columns=["NOME_UG", "NOME_PARTICIPANTE", "NOME_VENCEDOR", "MODALIDADE_COMPRA", "DESCRICAO_ITEM_COMPRA", "SITUACAO_LICITACAO", "MUNICIPIO", "RESULTADO_COMPRA_MES", "RESULTADO_COMPRA_ANO", "LICITACAO_CONLUIO", "QTD_PARTICIPANTES_ITEM", "OUTLIER", "COEFICIENTE_VARIACAO_ITEM", "CURTOSE_ITEM", "ASSIMETRIA"])
df_u.head()

## Experimentos - seleção de abordagem (seleção de atributos e balanceamento)

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import matthews_corrcoef

In [ ]:
def executar_modelo(name, estat, clf, X_train2, X_test2):
    print("Classificação do modelo: ", name, ". Variáveis estatísticas: ",  estat)
    clf.fit(X_train2, y_train)
    y_pred = clf.predict(X_test2).round()
    #Métricas gerais
    mcc = matthews_corrcoef(y_test, y_pred)
    print('Matthews correlation coefficient (MCC):', mcc)
    print(classification_report(y_pred, y_test))
    return clf

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
def gera_curva_roc(name, clf_ce, clf_se):
    _, ax = plt.subplots() 
    plt.plot([0, 1], [0, 1], 'k--', label='Classificador aleatório')
    RocCurveDisplay.from_estimator(clf_ce, X_test, y_test, ax=ax, name=name+" (COM var. estat.)")
    RocCurveDisplay.from_estimator(clf_se, X_test.iloc[:,0:5], y_test, ax=ax, name=name+" (SEM var. estat.)")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Positivos Falsos')
    plt.ylabel('Taxa de Positivos Verdadeiros')
    plt.legend()
    plt.show()

In [ ]:
import shap
def explicar_modelo_shap (name, clf):
    print('Explicação do modelo com SHAP')
    shap.initjs()
    X_train_summary = shap.kmeans(X_train, 3)
    explainer = shap.KernelExplainer(clf.predict, X_train_summary)   
    shap_values = explainer.shap_values(X_test, nsamples=1000)
    figure = plt.figure()
    shap.summary_plot(shap_values, X_test)

In [ ]:
import shap
def explicar_modelo_shap_arvore (name, clf):
    print('Explicação do modelo com SHAP')
    explainer = shap.TreeExplainer(clf)
    shap_values = explainer.shap_values(X_test)
    shap.initjs()
    shap.summary_plot(shap_values, X_test)

In [ ]:
import shap
def explicar_modelo_shap_linear (name, clf):
    print('Explicação do modelo com SHAP')
    explainer = shap.LinearExplainer(clf, X_train)
    shap_values = explainer.shap_values(X_test)
    shap.initjs()
    shap.summary_plot(shap_values, X_test)

### Codificação, normalização e partição

In [ ]:
X = df_u.drop(columns=['LICITACAO_CONLUIO'], axis=1)
y = df_u['LICITACAO_CONLUIO']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
import collections
print(collections.Counter(y_train))
print(collections.Counter(y_test))

In [ ]:
#Codifica
import category_encoders as ce
encoder = ce.OrdinalEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
encoder = ce.LeaveOneOutEncoder(return_df=True)
X_train = encoder.fit_transform(X_train, y_train)
X_test = encoder.transform(X_test)

In [ ]:
#Normaliza
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#Escala treinamento
scaler.fit(X_train)
scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(scaled, columns=X_train.columns)
#Escala teste
scaler.fit(X_test)
scaled = scaler.fit_transform(X_test)
X_test = pd.DataFrame(scaled, columns=X_test.columns)
#Escala treinamento y
scaler.fit(y_train)
scaled = scaler.fit_transform(y_train)
y_train = pd.DataFrame(scaled, columns=y_train.columns)
#Escala teste y
scaler.fit(y_test)
scaled = scaler.fit_transform(y_test)
y_test = pd.DataFrame(scaled, columns=y_test.columns)

### Seleção de atributos

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, RFE
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#Imprimi gráfico - Chi2
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(X_train.iloc[:,0:9], y_train)
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

In [ ]:
#Imprimi gráfico - Seleção univariada
fs = SelectKBest(score_func=f_classif, k=5)
fs.fit(X_train.iloc[:,0:9], y_train)
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

In [ ]:
gc.collect()

In [ ]:
#Imprimi gráfico - Mutual information
fs = SelectKBest(score_func=mutual_info_classif, k=5)
fs.fit(X_train.iloc[:,0:9], y_train)
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

In [ ]:
#Imprimi ranking - RFE
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
rfe.fit(X_train.iloc[:,0:9], y_train)
for i in range(X_train.iloc[:,0:9].shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

In [ ]:
#Definir a estratégia de seleção de atributos
smode = 'Chi2'
X2 = X_train
if (smode == 'Chi2'): #Chi2
    selector = SelectKBest(chi2, k=5)
    X2 = selector.fit_transform(X_train.iloc[:,0:9], y_train)
elif (smode == 'SU'): #Seleção univariada
    selector = SelectKBest(f_classif, k=5)
    X2 = selector.fit_transform(X_train.iloc[:,0:9], y_train)
elif (smode == 'MI'): #Mutual information
    selector = SelectKBest(mutual_info_classif, k=5)
    X2 = selector.fit_transform(X_train.iloc[:,0:9], y_train)    
    mutual_info_classif
elif (smode == 'RFE'): #Recursive Feature Elimination
    selector = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
    X2 = selector.fit_transform(X_train.iloc[:,0:9], y_train)
elif (smode == 'PCA'): #PCA
    from sklearn.decomposition import PCA
    selector = PCA(n_components=5)
    X2 = selector.fit_transform(X_train.iloc[:,0:9], y_train)

In [ ]:
#Nome das colunas selecionadas
X2_columns = X_train.iloc[:,0:9].columns
cols_sel = X2_columns[selector.get_support()]
print("Colunas selecionadas: ", cols_sel)
cols_excl = list(set(X_train.iloc[:,0:9].columns)-set(cols_sel))
print("Colunas excluídas: ", cols_excl)

In [ ]:
X_train.drop(columns=cols_excl, inplace=True)
X_test.drop(columns=cols_excl, inplace=True)

In [ ]:
del X2
del X2_columns
gc.collect()

### Codificação e balanceamento

In [ ]:
#Definir a estratégia de balanceamento
smode = 'smote'
if (smode == 'under'):
    from imblearn.under_sampling import RandomUnderSampler
    samp = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')
elif (smode == 'over'):
    from imblearn.over_sampling import RandomOverSampler
    samp = RandomOverSampler(sampling_strategy='minority')
elif (smode == 'smote'):
    from imblearn.over_sampling import SMOTE
    samp = SMOTE()
elif (smode == 'smoteenn'):
    from imblearn.combine import SMOTEENN
    samp = SMOTEENN(random_state=42)  
if (smode != 'na'):
    X_train, y_train = samp.fit_resample(X_train, y_train)

## Aprendizado

In [ ]:
import time

In [ ]:
n = gc.collect()
print("Número de objetos coletados pela função GC:", n)
print("Objetos restantes:", gc.garbage)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf_ce = executar_modelo("AdaBoost", "SIM", AdaBoostClassifier(), X_train, X_test)
clf_se = executar_modelo("AdaBoost", "NÃO", AdaBoostClassifier(), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("AdaBoost", "SOMENTE", AdaBoostClassifier(), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("AdaBoost", clf_ce, clf_se)
#inicio = time.time()
#explicar_modelo_shap_arvore ("AdaBoost", clf_ce)
#print(time.time() - inicio)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf_ce = executar_modelo("Árvore de decisão", "SIM", DecisionTreeClassifier(class_weight='balanced', max_depth=5), X_train, X_test)
clf_se = executar_modelo("Árvore de decisão", "NÃO", DecisionTreeClassifier(class_weight='balanced', max_depth=5), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("Árvore de decisão", "SOMENTE", DecisionTreeClassifier(class_weight='balanced', max_depth=5), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("Árvore de decisão", clf_ce, clf_se)
#inicio = time.time()
#explicar_modelo_shap_arvore ("Árvore de decisão", clf_ce)
#print(time.time() - inicio)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
clf_ce = executar_modelo("Extra Trees", "SIM", ExtraTreesClassifier(class_weight='balanced'), X_train, X_test)
clf_se = executar_modelo("Extra Trees", "NÃO", ExtraTreesClassifier(class_weight='balanced'), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("Extra Trees", "SOMENTE", ExtraTreesClassifier(class_weight='balanced'), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("Extra Trees", clf_ce, clf_se)
inicio = time.time()
explicar_modelo_shap ("Extra Trees", clf_ce)
print(time.time() - inicio)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf_ce = executar_modelo("Gradient Boosting", "SIM", GradientBoostingClassifier(), X_train, X_test)
clf_se = executar_modelo("Gradient Boosting", "NÃO", GradientBoostingClassifier(), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("Gradient Boosting", "SOMENTE", GradientBoostingClassifier(), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("Gradient Boosting", clf_ce, clf_se)
inicio = time.time()
explicar_modelo_shap_arvore ("Gradient Boosting", clf_ce)
print(time.time() - inicio)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf_ce = executar_modelo("K-NN", "SIM", KNeighborsClassifier(n_neighbors=5, n_jobs=-1), X_train, X_test)
clf_se = executar_modelo("K-NN", "NÃO", KNeighborsClassifier(n_neighbors=5, n_jobs=-1), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("K-NN", "SOMENTE", KNeighborsClassifier(n_neighbors=5, n_jobs=-1), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("K-NN", clf_ce, clf_se)
inicio = time.time()
explicar_modelo_shap ("K-NN", clf_ce)
print(time.time() - inicio)

In [ ]:
from sklearn.linear_model import Lasso
clf_ce = executar_modelo("Lasso", "SIM", Lasso(alpha=0.1), X_train, X_test)
clf_se = executar_modelo("Lasso", "NÃO", Lasso(alpha=0.1), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("Lasso", "SOMENTE", Lasso(alpha=0.1), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("Lasso", clf_ce, clf_se)
#inicio = time.time()
#explicar_modelo_shap_linear ("Lasso", clf_ce)
#print(time.time() - inicio)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_ce = executar_modelo("Regressão Logística", "SIM", LogisticRegression(class_weight='balanced'), X_train, X_test)
clf_se = executar_modelo("Regressão Logística", "NÃO", LogisticRegression(class_weight='balanced'), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("Regressão Logística", "SOMENTE", LogisticRegression(class_weight='balanced'), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("Regressão Logística", clf_ce, clf_se)
inicio = time.time()
explicar_modelo_shap_linear ("Regressão Logística", clf_ce)
print(time.time() - inicio)

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf_ce = executar_modelo("Naive Bayes (Gaussiano)", "SIM", GaussianNB(), X_train, X_test)
clf_se = executar_modelo("Naive Bayes (Gaussiano)", "NÃO", GaussianNB(), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("Naive Bayes (Gaussiano)", "SOMENTE", GaussianNB(), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("Naive Bayes (Gaussiano)", clf_ce, clf_se)
#inicio = time.time()
#explicar_modelo_shap ("Naive Bayes (Gaussiano)", clf_ce)
#print(time.time() - inicio)

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf_ce = executar_modelo("Naive Bayes (Bernoulli)", "SIM", BernoulliNB(), X_train, X_test)
clf_se = executar_modelo("Naive Bayes (Bernoulli)", "NÃO", BernoulliNB(), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("Naive Bayes (Bernoulli)", "SOMENTE", BernoulliNB(), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("Naive Bayes (Bernoulli)", clf_ce, clf_se)
#inicio = time.time()
#explicar_modelo_shap ("Naive Bayes (Bernoulli)", clf_ce)
#print(time.time() - inicio)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf_ce = executar_modelo("MLP", "SIM", MLPClassifier(), X_train, X_test)
clf_se = executar_modelo("MLP", "NÃO", MLPClassifier(), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("MLP", "SOMENTE", MLPClassifier(), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("MLP", clf_ce, clf_se)
inicio = time.time()
explicar_modelo_shap_deep ("Neural Net", clf_ce)
print(time.time() - inicio)

In [ ]:
#from sklearn.gaussian_process import GaussianProcessClassifier
#inicio = time.time()
#clf_ce = executar_modelo("Gaussian Process", "SIM", GaussianProcessClassifier(), X_train, X_test)
#clf_se = executar_modelo("Gaussian Process", "NÃO", GaussianProcessClassifier(), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
#gera_curva_roc("Gaussian Process", clf_ce, clf_se)
#explicar_modelo_shap ("Gaussian Process", clf_ce)
#print(time.time() - inicio)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
clf_ce = executar_modelo("QDA", "SIM", QuadraticDiscriminantAnalysis(), X_train, X_test)
clf_se = executar_modelo("QDA", "SOMENTE", QuadraticDiscriminantAnalysis(), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("QDA", "NÃO", QuadraticDiscriminantAnalysis(), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("QDA", clf_ce, clf_se)
#inicio = time.time()
#explicar_modelo_shap ("Quadratic Discriminant Analysis", clf_ce)
#print(time.time() - inicio)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_ce = executar_modelo("Random Forest", "SIM", RandomForestClassifier(class_weight='balanced'), X_train, X_test)
clf_se = executar_modelo("Random Forest", "NÃO", RandomForestClassifier(class_weight='balanced'), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("Random Forest", "SOMENTE", RandomForestClassifier(class_weight='balanced'), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("Random Forest", clf_ce, clf_se)
inicio = time.time()
#explicar_modelo_shap_arvore ("Random Forest", clf_ce)
#print(time.time() - inicio)

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
clf_ce = executar_modelo("Ridge", "SIM", RidgeClassifierCV(), X_train, X_test)
clf_se = executar_modelo("Ridge", "NÃO", RidgeClassifierCV(), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("Ridge", "SOMENTE", RidgeClassifierCV(), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("Ridge", clf_ce, clf_se)
inicio = time.time()
explicar_modelo_shap_linear ("Ridge", clf_ce)
print(time.time() - inicio)

In [ ]:
from sklearn.linear_model import SGDClassifier
clf_ce = executar_modelo("SGD", "SIM", SGDClassifier(class_weight='balanced'), X_train, X_test)
clf_se = executar_modelo("SGD", "NÃO", SGDClassifier(class_weight='balanced'), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("SGD", "SOMENTE", SGDClassifier(class_weight='balanced'), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("SGD", clf_ce, clf_se)
inicio = time.time()
explicar_modelo_shap_linear ("SGD", clf_ce)
print(time.time() - inicio)

In [ ]:
from sklearn import svm
clf_ce = executar_modelo("SVM", "SIM", svm.SVC(class_weight='balanced'), X_train, X_test)
clf_se = executar_modelo("SVM", "NÃO", svm.SVC(class_weight='balanced'), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("SVM", "SOMENTE", svm.SVC(class_weight='balanced'), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("SVM", clf_ce, clf_se)
inicio = time.time()
explicar_modelo_shap ("SVM", clf_ce)
print(time.time() - inicio)

In [ ]:
from xgboost import XGBClassifier
clf_ce = executar_modelo("XGBoost", "SIM", XGBClassifier(), X_train, X_test)
clf_se = executar_modelo("XGBoost", "NÃO", XGBClassifier(), X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = executar_modelo("XGBoost", "SOMENTE", XGBClassifier(), X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("XGBoost", clf_ce, clf_se)
inicio = time.time()
explicar_modelo_shap_arvore ("XGBoost", clf_ce)
print(time.time() - inicio)

In [ ]:
def treina_OneClassSVM (estat, X_train2, X_test2):
    print("Classificação do modelo: OneClassSVM. Variáveis estatísticas: ", estat)
    #Só treina com os dados normais
    #X_train_t = X_train2[y_train['LICITACAO_CONLUIO'] == 0.0]
    X_train_t = X_train2[y_train['LICITACAO_CONLUIO'] == 1.0]
    clf = OneClassSVM(kernel='rbf', gamma=0.1, nu=0.1)
    clf.fit(X_train_t)
    y_pred = clf.predict(X_test2)
    #Modelo classifica -1 para anômolos e 1 para normais, ou seja, precisa inverter
    #y_test_t = y_test.replace(1.0, -1.0)
    #y_test_t = y_test_t.replace(0.0, 1.0)
    y_test_t = y_test.replace(0.0, -1.0)
    print(classification_report(y_pred, y_test_t))
    mcc = matthews_corrcoef(y_test_t, y_pred)
    print('Matthews correlation coefficient (MCC):', mcc)

In [ ]:
from sklearn.svm import OneClassSVM
clf_ce = treina_OneClassSVM("SIM", X_train, X_test)
clf_se = treina_OneClassSVM("NÃO", X_train.iloc[:,0:5], X_test.iloc[:,0:5])
clf_e = treina_OneClassSVM("SOMENTE", X_train.iloc[:,6:], X_test.iloc[:,6:])
#gera_curva_roc("OneClassSVM", clf_ce, clf_se)
#inicio = time.time()
#explicar_modelo_shap ("OneClassSVM", clf_ce)
#print(time.time() - inicio)

## Feature importance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Abordagem para algoritmos que não fornecem importância de características
from sklearn.inspection import permutation_importance
def feature_importance_permutation (model_name, model):
    model.fit(X_train, y_train)
    result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
    #Ordenar importância
    sorted_idx = result.importances_mean.argsort()
    #Plotar importância
    plt.barh(np.array(X_train.columns)[sorted_idx], result.importances_mean[sorted_idx], align='center', height=0.5)
    plt.yticks(range(len(sorted_idx)), np.array(X_train.columns)[sorted_idx])
    plt.xlabel('Importância da permutação')
    plt.title('Importância dos atributos ('+model_name+')')
    plt.show()

In [ ]:
#Abordagem para algoritmos que fornecem importância de características por padrão, com atributo coef (tipicamente modelos lineares)
def feature_importance_linear (model_name, model):
    model.fit(X_train, y_train)
    coefficients = np.abs(model.coef_)
    #Normalizar importância
    coefficients = coefficients / coefficients.sum()
    #Ordenar importância
    sorted_idx = np.argsort(coefficients)
    #Plotar importância
    plt.barh(np.array(X_train.columns)[sorted_idx], coefficients[sorted_idx], align='center', height=0.5)
    plt.yticks(range(len(sorted_idx)), np.array(X_train.columns)[sorted_idx])
    plt.xlabel('Importância dos atributos')
    plt.title('Importância dos atributos ('+model_name+')')
    plt.show()

In [ ]:
from sklearn.neural_network import MLPClassifier
inicio = time.time()
feature_importance_permutation ('MLP', MLPClassifier())
print(time.time() - inicio)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
inicio = time.time()
feature_importance_permutation ('K-NN', KNeighborsClassifier(n_neighbors=5, n_jobs=-1))
print(time.time() - inicio)

In [ ]:
from sklearn.linear_model import LogisticRegression
inicio = time.time()
feature_importance_linear ('Regressão Logística', LogisticRegression(class_weight='balanced'))
print(time.time() - inicio)

In [ ]:
from sklearn.linear_model import SGDClassifier
inicio = time.time()
feature_importance_linear ('SGD', SGDClassifier(class_weight='balanced'))
print(time.time() - inicio)

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
inicio = time.time()
feature_importance_linear ('Ridge', RidgeClassifierCV())
print(time.time() - inicio)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
inicio = time.time()
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
feature_importance = model.feature_importances_
#Normalizar importância
feature_importance = feature_importance / feature_importance.sum()
#Ordenar importância
sorted_idx = np.argsort(feature_importance)
#Plotar importância
plt.barh(np.array(X_train.columns)[sorted_idx], feature_importance[sorted_idx], align='center', height=0.5)
plt.yticks(range(len(sorted_idx)), np.array(X_train.columns)[sorted_idx])
plt.xlabel('Importância dos atributos')
plt.title('Importância dos atributos (Extra Tree)')
plt.show()
print(time.time() - inicio)

In [ ]:
import xgboost as xgb
inicio = time.time()
model = xgb.XGBClassifier(class_weight='balanced')
model.fit(X_train, y_train)
#Plotar importância
xgb.plot_importance(model, importance_type='weight', grid=False, show_values=False, height=0.5)
plt.xlabel('Importância dos atributos')
plt.title('Importância dos atributos (XGBoost)')
plt.show()
print(time.time() - inicio)

## Explicações com LIME

In [ ]:
from lime.lime_tabular import LimeTabularExplainer
def explicar_modelo_lime (clf, y_pred, y_pred_prob, X_train, X_test, X_columns, i):
    print('Explicação do modelo com LIME')
    print('Instância:', i)
    print('Predict: ', y_pred[i])
    print('Predict proba: ', y_pred_prob[i])
    class_names = ['NAO', 'SIM'] 
    explainer = LimeTabularExplainer(X_train.to_numpy(), feature_names=X_columns, class_names=class_names, verbose=True, discretize_continuous=True, mode='classification')
    exp = explainer.explain_instance(X_test.iloc[i], clf.predict_proba, num_features=10)
    fig = exp.as_pyplot_figure(label=1)
    #exp.show_in_notebook(show_table=True)
    df = pd.DataFrame(exp.as_list(),columns=['Atributo','Contribuição'])
    display(df)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
clf = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test).round()
y_pred_prob = clf.predict_proba(X_test)
#Métricas gerais
mcc = matthews_corrcoef(y_test, y_pred)
print('Matthews correlation coefficient (MCC):', mcc)
print(confusion_matrix(y_test, y_pred))
print(collections.Counter(y_pred))
print(y_pred)

In [ ]:
print(X_test.shape[0])
for index, row in X_test.iterrows():
    i = X_test.index.get_loc(index)
    if (y_pred[i] == 1.0):
        print("i: ", i, "pred: ", y_pred[i], "conl:", y_test.iloc[i])

In [ ]:
print(X_test.shape[0])
for index, row in X_test.iterrows():
    i = X_test.index.get_loc(index)
    if (y_pred[i] == 0.0 and y_test['LICITACAO_CONLUIO'].iloc[i] == 1.0):
        print("i: ", i, "pred: ", y_pred[i], "conl:", y_test.iloc[i])

In [ ]:
explicar_modelo_lime (clf, y_pred, y_pred_prob, X_train, X_test, X_train.columns, 1099)

In [ ]:
explicar_modelo_lime (clf, y_pred, y_pred_prob, X_train, X_test, X_train.columns, 2899)

In [ ]:
explicar_modelo_lime (clf, y_pred, y_pred_prob, X_train, X_test, X_train.columns, 282873)

In [ ]:
explicar_modelo_lime (clf, y_pred, y_pred_prob, X_train, X_test, X_train.columns, 1)